In [6]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train = pd.read_csv('../../data/train.tsv', sep='\t')
test = pd.read_csv('../../data/test.tsv', sep='\t')

train = np.array(train[:50])
train_sentiment = train[:,-1]
train = train[:, :3]
test = np.array(test[:50])
print train.shape
print test.shape

train_test_split = len(train)
print train_test_split

data = np.concatenate((train, test), axis=0)
print data.shape
columns = ['PhraseId', 'SentenceId', 'Phrase']
data= pd.DataFrame(data, columns=columns)


(50, 3)
(50, 3)
50
(100, 3)


In [29]:
print data.columns

Index([u'PhraseId', u'SentenceId', u'Phrase'], dtype='object')


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

v1 = CountVectorizer(stop_words='english', min_df=2, max_df=0.95, max_features=1000)
v1.fit(data.Phrase)

# X1_train_train = v1.fit_transform(train_train.Phrase)
# X1_train_test = v1.fit_transform(train_test.Phrase)
# X1_test = v1.fit_transform(test.Phrase)

In [4]:
from textblob import TextBlob
def extract_verbs_and_adj(phrase):
    words = []
    
    tags = TextBlob(phrase).tags
    for tag in tags:
        if tag[1][:2] == "VB" or tag[1][:2] == "JJ":
            words.append(tag[0])
    return words

def looping_extract_verbs_and_adj(data_set):
    phrases_vb_adj = []

    for (index, phrase) in enumerate(data_set):
        if index % 10 == 0:
            print index
        phrases_vb_adj.append(' '.join(extract_verbs_and_adj(phrase)))
    return phrases_vb_adj

data_vb_adj = looping_extract_verbs_and_adj(data.Phrase)

# train_train_phrases_vb_adj = looping_extract_verbs_and_adj(train_train.Phrase)
# train_test_phrases_vb_adj = looping_extract_verbs_and_adj(train_test.Phrase)
# test_phrases_vb_adj = looping_extract_verbs_and_adj(test.Phrase)

0
10
20
30
40
50
60
70
80
90


In [5]:
def add_vb_adj_sentiment(phrases_vb_adj):
    v2 = CountVectorizer(ngram_range=(0, 4),  min_df=2, max_df=0.95, max_features=1000)
    X2 = v2.fit_transform(phrases_vb_adj)
#     print X2.shape

    sentiments = []
    for feature in v2.get_feature_names():
        sentiment = TextBlob(feature).sentiment
        value = sentiment.polarity * sentiment.subjectivity
        sentiments.append(value)

#     print v2.get_feature_names()
#     print sentiments

    from scipy.sparse import csr_matrix

    X3 = csr_matrix(X2.shape)
    for index in range(X2.shape[0]):
        row = X2[index]
        for col in range(row.shape[1]):
            X3[index, col] = X2[index, col] * sentiments[col]

    from scipy.sparse import hstack
    X4 = hstack([X1, X3])
#     print X4.shape
    return X4

X = add_vb_adj_sentiment(data_vb_adj)

# X_train_train = add_vb_adj_sentiment(X1_train_train, train_train_phrases_vb_adj)
# X_train_test = add_vb_adj_sentiment(X1_train_test, train_test_phrases_vb_adj)
# X_test = add_vb_adj_sentiment(X1_test, test_phrases_vb_adj)


/home/zhecan/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [12]:
print X.shape
print type(X)
train = X[:50, :]
test = X[50:, :]
print train.shape
print test.shape

(100, 92)
<class 'scipy.sparse.coo.coo_matrix'>


TypeError: 'coo_matrix' object has no attribute '__getitem__'

In [2]:

split_indexs = int(len(X)/3)
train_test = train[: split_indexs]
train_train = train[split_indexs:]


In [11]:

# print X_train_train.shape
# print X_train_test.shape
# print X_test.shape

(134, 85)
(66, 38)
(200, 136)


In [9]:
from sklearn.svm import SVC
clf = SVC()
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)



clf.fit(X_train_train, train_train.Sentiment) 

y_train_test_predict = clf.predict(X_train_test)
y_test_predict = clf.predict(X_test)


train_output = pd.DataFrame({
    'PhraseId': train_test.PhraseId,
    'Predicted': y_train_test_predict,
    'Sentiment': train_test.Sentiment 
})



test_output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_test_predict
})

ValueError: X.shape[1] = 38 should be equal to 85, the number of features at training time